Problem statement

In this competition, you’ll simulate a ventilator connected to a sedated patient's lung. The best submissions will take lung attributes compliance and resistance into account.

If successful, you'll help overcome the cost barrier of developing new methods for controlling mechanical ventilators. This will pave the way for algorithms that adapt to patients and reduce the burden on clinicians during these novel times and beyond. As a result, ventilator treatments may become more widely available to help patients breathe.

Import

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib
import matplotlib.style
import matplotlib as mpl
mpl.style.use('classic')
from mpl_toolkits.mplot3d import Axes3D

Load

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

Read

In [ ]:
train = pd.read_csv("/kaggle/input/ventilator-pressure-prediction/train.csv")
test = pd.read_csv("/kaggle/input/ventilator-pressure-prediction/test.csv")
submission = pd.read_csv("/kaggle/input/ventilator-pressure-prediction/sample_submission.csv")

In [ ]:
train

In [ ]:
test

In [ ]:
submission

Analyse target

In [ ]:
sns.displot(train['pressure'])

In [ ]:
target = train['pressure']
print("Minimum value: ", target.min())
print("Maximum value: ", target.max())
print("Average value: ", target.mean())
print("Standard deviation: ", target.std())

Combine

In [ ]:
combi = train.drop(['pressure'], axis=1).append(test)
combi

Drop id

In [ ]:
combi.drop(['id'],axis=1, inplace=True)
combi

Heatmap

In [ ]:
corrmat = combi.corr()
f, ax = plt.subplots(figsize=(12, 9))
sns.heatmap(corrmat, vmax=.8, square=True);

Check for missing values

In [ ]:
combi.isnull().sum()

Normalise

In [ ]:
#combi = (combi - combi.min()) / (combi.max() - combi.min())
#combi

Standardise

In [ ]:
combi = (combi - combi.mean()) / np.std(combi)
combi

Define X and y

In [ ]:
y = target
X = combi[: len(train)]
X_test = combi[len(train) :]

y = np.array(y, dtype=np.float32)
X = np.array(X, dtype=np.float32)
X_test = np.array(X_test, dtype=np.float32)
X.shape, y.shape, X_test.shape

Graph

In [ ]:
# Visualize our data
import matplotlib.pyplot as plt
import numpy as np

x = X[:,3]
y = y

plt.scatter(x, y)
plt.show()

Import Pytorch

In [ ]:
#try:- https://www.analyticsvidhya.com/blog/2020/10/perform-regression-analysis-with-pytorch-seamlessly/
import torch
import torch.nn as nn 

Reshape

In [ ]:
X_tensor = torch.from_numpy(X.reshape(X.shape[0],X.shape[1]))
X_test_tensor = torch.from_numpy(X_test.reshape(X_test.shape[0],X_test.shape[1]))
y_tensor = torch.from_numpy(y.reshape(y.shape[0],1))

X_tensor.shape, y_tensor.shape, X_test_tensor.shape

In [ ]:
input_size = 6
output_size = 1

Linear regression

Define layer

In [ ]:
model = nn.Linear(input_size , output_size)

Define loss and optimizer

In [ ]:
learning_rate = 0.0001
l = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr =learning_rate )

Training

In [ ]:
num_epochs = 10000

for epoch in range(num_epochs):
    #forward feed
    y_pred = model(X_tensor.requires_grad_())

    #calculate the loss
    loss= l(y_pred, y_tensor)

    #backward propagation: calculate gradients
    loss.backward()

    #update the weights
    optimizer.step()

    #clear out the gradients from the last step loss.backward()
    optimizer.zero_grad()
    
    if (epoch+1)%100 == 0:
        print('epoch:', epoch+1,',loss=',loss.item())


Predict on X

In [ ]:
prediction = model(X_tensor).detach().numpy()
prediction

Graph

In [ ]:
plt.scatter(X_tensor[:,3].detach().numpy()[:100] , y_tensor.detach().numpy()[:100])
plt.plot(X_tensor[:, 3].detach().numpy()[:100] , prediction[:100] , "red")
plt.xlabel("X")
plt.ylabel("y")
plt.show()

In [ ]:
from sklearn.metrics import mean_squared_error

rmse = mean_squared_error(y_tensor.detach().numpy(), prediction, squared=False)
rmse

In [ ]:
compare = pd.DataFrame({'actual': y, 'predicted': prediction.ravel()})
compare

Predict on X_test

In [ ]:
test_prediction = model(X_test_tensor).detach().numpy()
test_prediction

Prepare submission

In [ ]:
submission['pressure'] = test_prediction
submission.to_csv('submission.csv',index=False) # writing data to a CSV file
submission = pd.read_csv("submission.csv")
submission
